In [ ]:
import os
import json
import requests
from dotenv import load_dotenv
load_dotenv('.env')

# Name of the container in your Blob Storage Datasource ( in credentials.env)
BLOB_CONTAINER_NAME = os.environ['container'] 
datasource_name=os.environ['BLOB_NAME'] 
print(' container : ',BLOB_CONTAINER_NAME)
print(' datasource : ',datasource_name)

In [ ]:
## ref : https://github.com/MSUSAzureAccelerators/Azure-Cognitive-Search-Azure-OpenAI-Accelerator/blob/main/01-Load-Data-ACogSearch.ipynb

# Setup the Payloads header
headers = {'Content-Type': 'application/json','api-key': os.environ['AZURE_SEARCH_KEY']}
params = {'api-version': os.environ['AZURE_SEARCH_API_VERSION']}
# print(' ==> ',os.environ['AZURE_SEARCH_KEY'])

In [ ]:
## ref: https://sehun.me/upload-files-to-azure-blob-storage-python-db1f122309b2
from azure.storage.blob import BlobServiceClient
from azure.storage.blob import BlobClient

### upload files in \\DOCS\\ to blob storage ( according to BLOB_CONNECTION_STRING and specified BLOB_CONTAINER_NAME )
if_upload_new_document=1
if(if_upload_new_document==1):
    print(' ---- Upload new document ----- ')
        
    # Set the connection string to your Azure Blob Storage account
    connection_string = os.environ['BLOB_CONNECTION_STRING']
    print(' conn : ',connection_string)
    # Set the name of the container where you want to upload the files
    local_folder_path = os.getcwd()+"\\DOCS\\"

    container_name = BLOB_CONTAINER_NAME

    # Set the local paths to the folders containing the files you want to upload
    local_folder_paths = [local_folder_path]

    # Create a BlobServiceClient object
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)

    # Create a ContainerClient object
    container_client = blob_service_client.get_container_client(container_name)

    blob_list = container_client.list_blobs()
    print("\n----------List of blobs in the container ----------", blob_list)
    for blob in blob_list:
        print("----> " , blob)  ##.name)

    # Loop through all the local folders
    for local_folder_path in local_folder_paths:
        # Loop through all the files in the local folder
        for root, _, files in os.walk(local_folder_path):
            for file in files:
                print(' file : ',file,'  :: ',type(file))
                # Get the local path and name of the file
                local_file_path = os.path.join(root, file)
                blob_name = os.path.relpath(local_file_path, local_folder_path).replace("\\", "/")
                # print(' blob name : ',blob_name,' -- ',type(blob_name))
                # Create a BlobClient object for the file
                blob_client = container_client.get_blob_client(blob=blob_name)

                # Upload the file to Azure Blob Storage
                with open(local_file_path, "rb") as data:
                    print('  file ',data,' -- ',type(data))    
                    blob_client.upload_blob(data, overwrite=True)   ### upload to container
      


In [ ]:
from azure.storage.blob import BlobServiceClient

def download_blob(storage_connection_string, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(storage_connection_string)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    
    with open(blob_name, "wb") as file:
        file.write(blob_client.download_blob().readall())

### Download FILES from BLOB Storage based on spcified filenames
if_downloadfile_from_storage=0
if(if_downloadfile_from_storage==1):

    # Set the connection string, container name, blob name
    storage_connection_string = os.environ['BLOB_CONNECTION_STRING']

    # Create a BlobServiceClient object
    blob_service_client = BlobServiceClient.from_connection_string(storage_connection_string)
    # blob_service_client = BlobClient.from_connection_string(connection_string, container_name=BLOB_CONTAINER_NAME, blob_name=Blob_Name)

    # Create a ContainerClient object
    container_client = blob_service_client.get_container_client(container_name)
    blob_list = container_client.list_blobs()
    print("\n----------List of blobs in the container ----------", blob_list)
    filelist=[]
    for blob in blob_list:
        print("----> " , blob.name)
        filelist.append(blob.name)

    print(' file : ',filelist)

    container_name = BLOB_CONTAINER_NAME
    for fileid in filelist[:1]:
        blob_name = fileid  # 'A_Framework_for_Delineating_the_Scale_Ex.pdf'
        # Download the blob
        download_blob(storage_connection_string, container_name, blob_name)